In [1]:
# Import pandas for data handling
import pandas as pd

# NLTK is our Natural-Language-Took-Kit
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Libraries for helping us with strings
import string
# Regular Expression Library
import re

# Import our text vectorizers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


# Import our classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


# Import some ML helper function
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report


# Import our metrics to evaluate our model
from sklearn import metrics
from sklearn.metrics import classification_report


# Library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# You may need to download these from nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# stopwords = stopwords.words('english')

In [2]:
df=pd.read_csv(r'data/DS_DA_BS.csv')
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,role
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1,DS
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1,DS
2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,TRUE,DS
3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1,DS
4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1,DS


In [3]:
df["Salary Estimate"].value_counts()

$93K-$151K (Glassdoor est.)         62
$60K-$124K (Glassdoor est.)         62
$113K-$180K (Glassdoor est.)        61
$59K-$85K (Glassdoor est.)          61
$43K-$76K (Glassdoor est.)          61
                                    ..
$10-$26 Per Hour(Glassdoor est.)     3
$57K-$70K (Glassdoor est.)           2
-1                                   1
$41K-$74K (Glassdoor est.)           1
$59K-$98K (Glassdoor est.)           1
Name: Salary Estimate, Length: 366, dtype: int64

In [4]:
# utitility functions from jerry
#using re to extract numbers from the string
def extract_numbers(arr):#arr is the array of strings
    min_result=[]
    max_result=[]
#     result=[]
    for i in arr:
        temp=re.findall('[0-9]+',i)
        if len(temp)==2:
            min_result.append(int(temp[0]))
            max_result.append(int(temp[1]))
        else:
            min_result.append(int(temp[0]))
            max_result.append(int(temp[0]))
    return min_result,max_result  

def extract_numbers_max(x):
    temp=re.findall(r'[0-9]+',x)
    return max([int(i) for i in temp])

def extract_numbers_min(x):
    temp=re.findall(r'[0-9]+',x)
    return min([int(i) for i in temp])

In [5]:
# get median salary
df['min salary']=df['Salary Estimate'].apply(extract_numbers_min)
df['max salary']=df['Salary Estimate'].apply(extract_numbers_max)
df['Median Salary'] = (df['min salary'] + df['max salary'])/2
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,role,min salary,max salary,Median Salary
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1,DS,111,181,146.0
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1,DS,111,181,146.0
2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,TRUE,DS,111,181,146.0
3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1,DS,111,181,146.0
4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1,DS,111,181,146.0


In [6]:
# find out rows with hourly pay instead of salary and rows with a -1 salary
c1 = ~df['Salary Estimate'].str.contains("Hour")
c2 = ~(df['Salary Estimate'] == '-1')
df = df[c1 & c2]
series = df["Salary Estimate"].value_counts()
for i in range(len(series)):
    print(series.index[i], series.values[i])

$93K-$151K (Glassdoor est.) 62
$60K-$124K (Glassdoor est.) 62
$43K-$76K (Glassdoor est.) 61
$113K-$180K (Glassdoor est.) 61
$59K-$85K (Glassdoor est.) 61
$71K-$122K (Glassdoor est.) 60
$41K-$79K (Glassdoor est.) 60
$48K-$87K (Glassdoor est.) 60
$43K-$78K (Glassdoor est.) 60
$111K-$175K (Glassdoor est.) 60
$74K-$140K (Glassdoor est.) 60
$35K-$65K (Glassdoor est.) 60
$49K-$92K (Glassdoor est.) 59
$41K-$80K (Glassdoor est.) 59
$49K-$91K (Glassdoor est.) 59
$74K-$123K (Glassdoor est.) 59
$39K-$69K (Glassdoor est.) 58
$76K-$122K (Glassdoor est.) 58
$42K-$76K (Glassdoor est.) 57
$41K-$78K (Glassdoor est.) 57
$38K-$68K (Glassdoor est.) 57
$57K-$104K (Glassdoor est.) 57
$53K-$99K (Glassdoor est.) 56
$55K-$101K (Glassdoor est.) 55
$43K-$84K (Glassdoor est.) 53
$37K-$75K (Glassdoor est.) 53
$38K-$59K (Glassdoor est.) 52
$43K-$79K (Glassdoor est.) 51
$85K-$159K (Glassdoor est.) 50
$42K-$80K (Glassdoor est.) 50
$55K-$103K (Glassdoor est.) 47
$69K-$127K (Glassdoor est.) 46
$119K-$147K (Glassdoor es

In [9]:
# function to add column to classify salaries into ranges
def get_label(salary, salary_range=20):
    start = 0
    while salary > start:
        start += salary_range
    return str(start - salary_range) + " to " + str(start) + "(thousands)"

In [11]:
df['Salary Labels']=df['Median Salary'].apply(get_label)
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,role,min salary,max salary,Median Salary,Salary Labels
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1,DS,111,181,146.0,140 to 160(thousands)
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1,DS,111,181,146.0,140 to 160(thousands)
2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,TRUE,DS,111,181,146.0,140 to 160(thousands)
3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1,DS,111,181,146.0,140 to 160(thousands)
4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1,DS,111,181,146.0,140 to 160(thousands)
